In [1]:
import vtk
from PIL import Image
import os

pngpath = "f:\\GitHub\\pyaxisvm\\working\\1000.png"
filename, extension  = os.path.splitext(pngpath)
pmgfile = "{}.pgm".format(filename)
with Image.open(pngpath) as im:
    im.save(pmgfile)

imageIn = vtk.vtkPNMReader()
imageIn.SetFileName(pmgfile)

gaussian = vtk.vtkImageGaussianSmooth()
gaussian.SetStandardDeviations(2, 2)
gaussian.SetDimensionality(2)
gaussian.SetRadiusFactors(1, 1)
gaussian.SetInputConnection(imageIn.GetOutputPort())

geometry = vtk.vtkImageDataGeometryFilter()
geometry.SetInputConnection(gaussian.GetOutputPort())

aClipper = vtk.vtkClipPolyData()
aClipper.SetInputConnection(geometry.GetOutputPort())
aClipper.SetValue(130)
aClipper.GenerateClipScalarsOff()
aClipper.InsideOutOn()
aClipper.GetOutput().GetPointData().CopyScalarsOff()
aClipper.Update()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(aClipper.GetOutputPort())
mapper.ScalarVisibilityOff()

letter = vtk.vtkActor()
letter.SetMapper(mapper)

In [2]:
def transformPolyData(actor):
    polyData = vtk.vtkPolyData()
    polyData.DeepCopy(actor.GetMapper().GetInput())
    transform = vtk.vtkTransform()
    transform.SetMatrix(actor.GetMatrix())
    fil = vtk.vtkTransformPolyDataFilter()
    fil.SetTransform(transform)
    fil.SetInputDataObject(polyData)
    fil.Update()
    polyData.DeepCopy(fil.GetOutput())
    return polyData

In [3]:
import pyvista as pv
from dewloosh.mesh.topo.tr import Q4_to_T3
import numpy as np

from dewloosh.mesh.tri.triutils import edges_tri
from dewloosh.mesh.topo import unique_topo_data

polydata = pv.wrap(transformPolyData(letter))
mesh = pv.wrap(polydata).cast_to_unstructured_grid()

coords = np.array(mesh.points)
tri = []
cells = mesh.cells_dict
if 5 in cells:
    tri.append(mesh.cells_dict[5])
if 9 in cells:
    _, triQ4 = Q4_to_T3(coords, mesh.cells_dict[9])
    tri.append(triQ4)
topo = np.vstack(tri)

edges, edgeIDs = unique_topo_data(edges_tri(topo))
edges += 1
edgeIDs += 1

In [4]:
from axisvm.com.client import start_AxisVM
axvm = start_AxisVM(visible=True, daemon=True)

modelId = axvm.Models.New()
axm = axvm.Models.Item[modelId]
axm.Settings.EditingTolerance = -1
wdir = ""

from axisvm.com.tlb import RPoint3d
foo = lambda x : RPoint3d(x=x[0], y=x[1], z=x[2])
axm.BeginUpdate()
axm.Nodes.BulkAdd(list(map(foo, coords)))
axm.EndUpdate()

from axisvm.com.tlb import lgtStraightLine, RLineData

def gen_line(edge):
    return RLineData(
        NodeId1 = edge[0],
        NodeId2 = edge[1],
        GeomType = lgtStraightLine
    )
axm.BeginUpdate()
axm.Lines.BulkAdd(list(map(gen_line, edges)))
axm.EndUpdate()

from axisvm.com.tlb import vTop
axm.View = vTop
axm.FitInView();

from axisvm.com.tlb import ndcEuroCode
axm.Settings.NationalDesignCode = ndcEuroCode
matId = axm.Materials.AddFromCatalog(ndcEuroCode, "S235")

from axisvm.com.tlb import RSurfaceAttr, lnlTensionAndCompression, \
    RResistancesXYZ, schLinear, stShell, RElasticFoundationXYZ, \
    RNonLinearityXYZ, RSurface
    
SurfaceAttr = RSurfaceAttr(
    Thickness=1,
    SurfaceType=stShell,
    RefZId=0,
    RefXId=0,
    MaterialId=matId,
    ElasticFoundation=RElasticFoundationXYZ(0, 0, 0),
    NonLinearity=RNonLinearityXYZ(lnlTensionAndCompression,
                                  lnlTensionAndCompression,
                                  lnlTensionAndCompression),
    Resistance=RResistancesXYZ(0, 0, 0),
    Charactersitics=schLinear)

def gen_surface(edges):
    return RSurface(
        N=3,
        LineIndex1 = edges[0],
        LineIndex2 = edges[1],
        LineIndex3 = edges[2],
        Attr = SurfaceAttr,
        DomainIndex = 0
    )
axm.BeginUpdate()
axm.Surfaces.BulkAdd(list(map(gen_surface, edgeIDs)))
axm.EndUpdate()


0

In [24]:
axm.Surfaces.Count

180224